In [1]:
import sys
sys.path.append('/home/zss/estimation_new/baseline_new/')
import Utils
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import random
import os
import easygraph as eg
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
from easygraph.nn import HGNNConv
from easygraph.classes import Graph
from easygraph.nn import HyperGCNConv
import torch.nn.functional as F
from scipy.stats import kendalltau
import networkx as nx



def seed_torch(seed=42):
    seed = int(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = True

def generate_dict(matrix):
    # 初始化两个字典
    nd_he = {}
    he_nd = {}

    rows, cols = matrix.shape
    for row_idx in range(rows):
        for col_idx in range(cols):
            if matrix[row_idx, col_idx] == 1:
                # 更新 row_to_cols 字典
                if row_idx not in nd_he:
                    nd_he[row_idx] = []
                nd_he[row_idx].append(col_idx)

                # 更新 col_to_rows 字典
                if col_idx not in he_nd:
                    he_nd[col_idx] = []
                he_nd[col_idx].append(row_idx)
    return nd_he, he_nd

def get_neigbors(g, node, depth):
    output = {}
    layers = dict(nx.bfs_successors(g, source=node, depth_limit=depth))
    nodes = [node]
    for i in range(1, depth + 1):
        output[i] = []
        for x in nodes:
            output[i].extend(layers.get(x, []))
        nodes = output[i]
    return output


def H_index(g, node_set=-1):
    if node_set == -1:
        nodes = list(g.nodes())
        d = dict()
        H = dict()  # H-index
        for node in nodes:
            d[node] = g.degree(node)
        for node in nodes:
            neighbors = list(g.neighbors(node))
            neighbors_d = [d[x] for x in neighbors]
            for y in range(len(neighbors_d)):
                if y > len([x for x in neighbors_d if x >= y]):
                    break
            H[node] = y - 1

    if node_set in list(g.nodes()):  # 计算节点node的H-index
        neighbors = list(g.neighbors(node))
        neighbors_d = [d[x] for x in neighbors]
        for y in range(len(neighbors_d)):
            if y > len([x for x in neighbors_d if x >= y]):
                break
        H = y - 1
    return H


def k_shell(G, data_memory,train_nodes=[]):
    for x in data_memory: x[0] = int(x[0])
    simu_I = data_memory.copy()
    simu_I.sort(key=lambda x: x[1], reverse=True)
    simu_sort = [x[0] for x in simu_I]


    K = nx.core_number(G)
    k = [[x, K[x]] for x in K]
    k.sort(key=lambda x: x[1], reverse=True)
    k_sort = [x[0] for x in k if x[0] in simu_sort]

    for node in train_nodes: # test set Ken
        simu_sort.remove(node)
        k_sort.remove(node)

    node_rank_simu = list(range(1, len(simu_sort) + 1))
    node_rank_K = [k_sort.index(x) if x in k_sort else len(k_sort) for x in
                   simu_sort]
    ken_K = kendalltau(node_rank_simu, node_rank_K)
    K = list(K.values())
    # #计算肯德尔系数
    value_ = sorted(K, reverse=True)
    rank = [1]
    for i in range(1, len(value_)):
        if value_[i] < value_[i - 1]:
            rank.append(i + 1)
        elif value_[i] == value_[i - 1]:
            rank.append(rank[-1])
    # print(ken_K[0])
    return ken_K[0], rank,k_sort,node_rank_K


def future_generate(im,im_name):

    D = np.load(f'/home/zss/estimation_new/hyper_distance/{im_name}_distance_1.npy')

    A = np.dot(im, im.T)
    np.fill_diagonal(A, 0)
    adj_matrix = (A >= 1).astype(int)
    # print('complete adj_matrix')
    # 将邻接矩阵转换为边对列表
    edge_list = [(i, j) for i in range(len(adj_matrix)) for j in range(len(adj_matrix[i])) if adj_matrix[i][j] == 1]
    G = nx.Graph()
    G.add_edges_from(edge_list)

    # 保证所有节点都在图中，孤立节点也会被添加
    for i in range(len(adj_matrix)):
        if i not in G:
            G.add_node(i)

    input = torch.ones(im.shape[0], 3)

    deg = Utils.degree_centrality(im)
    print('finish:dcc',len(deg))
    h_index = list(H_index(G).values())
    print('finish:h-index',len(h_index))
    k_shell = list(nx.core_number(G))
    print('finish:k-shell',len(k_shell))

    input[:, 0] = torch.tensor(deg)
    input[:, 1] = torch.tensor(h_index)
    input[:, 2] = torch.tensor(k_shell)

    return input

class LSTMModel(nn.Module):
    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            hidden_size2: int,
            out_features: int
                 ):
        super(LSTMModel, self).__init__()
        # 定义LSTM层，输入特征为3，隐藏层单元数为128
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)#LSTM

        # self.lstm2 = nn.LSTM(hidden_size, hidden_size)#LSTM
        # self.lstm = nn.GRU(input_size, hidden_size, batch_first=True)#GRU
        self.dropout = nn.Dropout(p=0.5)
        # 定义全连接层，隐藏层到32个神经元
        self.l1 = nn.Linear(hidden_size, hidden_size2)
        # 定义输出层，将32个神经元连接到输出1个值
        self.l2 = nn.Linear(hidden_size2, out_features)

    def forward(self, x):
        x = x.unsqueeze(1)
        # LSTM层的输出
        lstm_out, _ = self.lstm(x)  # 忽略隐状态输出
        # 取LSTM输出的最后一个时间步的输出，维度是(batch_size, 128)
        lstm_out_last_step = lstm_out[:, -1, :]
        lstm_out_last_step = self.dropout(lstm_out_last_step)

        # lstm_out2, _ =self.lstm2(lstm_out_last_step.unsqueeze(1))
        # lstm_out_last_step2 = lstm_out2[:, -1, :]
        # lstm_out_last_step2 = self.dropout(lstm_out_last_step2)
        # 通过第一个全连接层
        l1_out = self.l1(lstm_out_last_step)
        # 通过第二个全连接层得到最终输出
        # l1_out = self.dropout(l1_out)
        output = F.relu(self.l2(l1_out))
        return output


def preprocess(hg, final_features, label, data):

# There is no default feature vector for this dataset. Users can generate their own features.
# Here we use random initialisation to generate 100-dimensional node feature vectors

  input_feature_dim = final_features.shape[1]
  hidden_dim = 128
  hidden_dim_2 = 32
  out_dim = 1
  
  '''
  Since there is no default split for this dataset, here we split the test set, validation set, and test set in a 50:25:25 fashion
  '''
  # 初始划分：90%的数据用于训练和验证，10%的数据用于测试
  train_val_nodes, test_nodes = train_test_split(hg.v, test_size=0.1, random_state=42)
# 在训练和验证数据集里再划分：80%的数据用于训练，20%的数据用于验证
  train_nodes, val_nodes = train_test_split(train_val_nodes, test_size=0.2, random_state=42)
  train_mask = train_nodes
  val_mask = val_nodes
  test_mask = test_nodes

  X = final_features.float()

  y = label.float()

  dataset = {}
  dataset["structure"] = eg.Hypergraph(num_v=label.shape[0], e_list=data)
  dataset["features"] = X
  dataset["labels"] = y
  dataset["train_mask"] = train_mask
  dataset["val_mask"] = val_mask
  dataset["test_mask"] = test_mask
  # dataset["num_classes"] = num_classes

  model = LSTMModel(input_size = input_feature_dim, hidden_size = hidden_dim, hidden_size2 = hidden_dim_2, out_features = out_dim)

  return dataset, model

def train(data: dict, model: nn.Module, optimizer: torch.optim.Optimizer, criterion: nn.Module):
    features, structure = data["features"], data["structure"]
    train_mask, labels = data["train_mask"], data["labels"]
    optimizer.zero_grad()
    outputs = model(features)
    loss = criterion(torch.log(outputs[train_mask]+1), torch.log(labels[train_mask]+1))
    # loss = criterion(outputs[train_mask], labels[train_mask])
    loss.backward()
    optimizer.step()
    return loss

torch.no_grad()
def valid(model, data, weights):
    features, labels, structure = data["features"], data["labels"], data["structure"]
    val_mask = data["val_mask"]
    weights = weights
    
    model.eval()
    with torch.no_grad():
        outputs = model(features)
        log_outputs = torch.log(outputs[val_mask] + 1)
        log_labels = torch.log(labels[val_mask] + 1)
        mse = nn.functional.mse_loss(log_outputs.squeeze(), log_labels).item()
    return mse

torch.no_grad()
def test(model, data, weights):
    features, structure = data["features"], data["structure"]
    test_mask, labels = data["test_mask"], data["labels"]
    weights = weights
    model.eval()
    with torch.no_grad():
        outputs = model(features)
        # criterion = WeightedMSELoss(weights[data["test_mask"]])
        log_outputs = torch.log(outputs[test_mask] + 1)
        log_labels = torch.log(labels[test_mask] + 1)
        mse = nn.functional.mse_loss(log_outputs.squeeze(), log_labels).item()
        # mse = criterion(outputs[test_mask].squeeze(), labels[test_mask])
    return mse


def draw_loss_curve(loss1 ,save_path = "loss_pic.png"):
    plt.clf()
    epochs = range(1, len(loss1) + 1)
    plt.plot(epochs, loss1, 'b', label='EG Training loss')
    plt.title('Training Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    if save_path is not None:
        plt.savefig(save_path)
    plt.show()

file_name=[
    'Algebra', 
    'Bars-Rev', 
    'Geometry', 
    'iAF1260b', 
    'iJO1366', 
    'Music-Rev', 
    # 'NDC-classes', 
    'Restaurants-Rev', 
    'senate-committees', 
    'fb-tvshow', 
    'HighSchool_2013_hour', 
    'Highschool_2012_hour', 
    'ht09_contact_hour', 
    'soc-hamsterster', 
    'house-committees'
]

In [ ]:
if __name__ == '__main__':
    kendalltau_value = [] 
    test_mse = []
    seed_torch(42)
    for name in file_name:
        matrix = np.load(f'/home/zss/estimation_new/datamatrix/{name}_incmatrix.npy')
        N, M = matrix.shape
        nd_he, he_nd = generate_dict(matrix)

        # 使用字典推导式来创建一个新的字典，过滤掉值只有一个元素的键值对
        filtered_dict = {k: v for k, v in he_nd.items() if len(v) > 1}

        data = [] 
        for key in filtered_dict.keys():
            data.append(filtered_dict[key])

        npy_array = np.load(f'/home/zss/estimation_new/spread_RP/{name}_ICRP_iter2000_time50.npy')
        label = torch.from_numpy(npy_array)

        hg = eg.Hypergraph(num_v = label.shape[0], e_list = data,  merge_op="sum")

        # final_features =  future_generate(matrix, name)
        #距离特征结合中心性特征
        center_feature=np.load(f'/home/zss/estimation_new/center_feature/{name}_center.npy')
        center_feature=torch.from_numpy(center_feature).to(torch.float32)
        

        #距离特征
        hydis = np.zeros((matrix.shape[0], matrix.shape[0]))
        order = 1
        for s in range(1,order+1):
            temp=np.load(f'/home/zss/estimation_new/hyper_distance/{name}_distance_{s}.npy')
            temp=1/temp
            hydis+=temp*((0.1)**(order-s))
            
        np.fill_diagonal(hydis,0)
        hydis=torch.from_numpy(hydis).to(torch.float32)

        final_features = torch.cat((hydis,center_feature), dim=1).to(torch.float32)

        
        dataset, model = preprocess(hg, final_features, label, data)
        loss_lst = []
        acc = []
        test_ls = []
        epoch = 500
        lr = 0.001
        # 示例：为较大的真实值样本赋予更高的权重
        weights = torch.sqrt(label)
        loss_fn = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr = lr)

        best_val_loss = float('inf')  # 初始化最小验证集损失为正无穷
        for i in range(epoch):
            model.train()
            loss = train(data = dataset, model = model, optimizer=optimizer, criterion=loss_fn)
            loss_lst.append(loss.detach().numpy())
            print(f"epoch: {i}, loss : {loss}")
        
            model.eval()
            val_acc = valid(model = model, data = dataset, weights=weights)
            acc.append(val_acc)
            if val_acc < best_val_loss:
                best_val_loss = val_acc
                torch.save(model.state_dict(), f'/home/zss/estimation_new/baseline_new/GLSTM/GLSTM_best_model/best_model_{name}.pth')  # 保存当前模型参数
            else:
                print(f'epoch {epoch + 1}, 验证集损失: {val_acc}, 未达到最佳')    

            

        print("Training finish!")
        
        # draw_loss_curve(loss_lst)
        # draw_loss_curve(acc)
        # draw_loss_curve(test_ls)

        features, labels, structure = dataset["features"], dataset["labels"], dataset["structure"]
        test_mask = dataset["test_mask"]
        train_mask = dataset["train_mask"]

        
        dataset, model_T = preprocess(hg, final_features, label, data)
        model_T.load_state_dict(torch.load(f'/home/zss/estimation_new/baseline_new/GLSTM/GLSTM_best_model/best_model_{name}.pth'))
        model_T.eval()
        with torch.no_grad():
            outputs = model_T(features)

        test_acc = test(model = model_T, data=dataset, weights=weights)
        test_mse.append(round(test_acc,6))
        
        
        y = labels[test_mask].squeeze()
        x = outputs[test_mask]
        # plt.xlabel('predict')
        # plt.ylabel('true')
        # plt.scatter(x.tolist(),y.tolist())
        # plt.title(f'{name} kendall:{round(kendalltau(x, y)[0],6)}')
        kendalltau_value.append(round(kendalltau(x, y)[0],6))
        # plt.savefig(f"/home/zss/estimation_new/centrality_version/figures_cp/{name}.jpg",dpi=600)
        # plt.clf()  # 或者 plt.close()

    np_data_1 = np.array(kendalltau_value)

    np_data_2 = np.array(test_mse)

    np.save('/home/zss/estimation_new/baseline_new/GLSTM/kendall_glstm.npy', np_data_1)

    np.save('/home/zss/estimation_new/baseline_new/GLSTM/MSE_glstm.npy', np_data_2)

    print(kendalltau_value)
    print(test_mse)

In [3]:
##### 获取测试结果，直接调用保存的模型

for name in file_name:
    seed_torch(42)
    matrix = np.load(f'/home/zss/estimation_new/datamatrix/{name}_incmatrix.npy')
    N, M = matrix.shape
    nd_he, he_nd = generate_dict(matrix)

    # 使用字典推导式来创建一个新的字典，过滤掉值只有一个元素的键值对
    filtered_dict = {k: v for k, v in he_nd.items() if len(v) > 1}

    data = [] 
    for key in filtered_dict.keys():
        data.append(filtered_dict[key])

    npy_array = np.load(f'/home/zss/estimation_new/spread_RP_threshold/{name}_ICRP_threshold_iter2000_time50.npy')
    label = torch.from_numpy(npy_array)

    hg = eg.Hypergraph(num_v = label.shape[0], e_list = data,  merge_op="sum")

    center_feature=np.load(f'/home/zss/estimation_new/center_feature/{name}_center.npy')
    center_feature=torch.from_numpy(center_feature).to(torch.float32)

    #距离特征
    hydis = np.zeros((matrix.shape[0], matrix.shape[0]))
    order = 1
    for s in range(1,order+1):
        temp=np.load(f'/home/zss/estimation_new/hyper_distance/{name}_distance_{s}.npy')
        temp=1/temp
        hydis+=temp*((0.1)**(order-s))
        
    np.fill_diagonal(hydis,0)
    hydis=torch.from_numpy(hydis).to(torch.float32)

    final_features = torch.cat((hydis,center_feature), dim=1).to(torch.float32)

    dataset, model_T = preprocess(hg, final_features, label, data)

    features, labels, structure = dataset["features"], dataset["labels"], dataset["structure"]
    test_mask = dataset["test_mask"]
    train_mask = dataset["train_mask"]

    model_T.load_state_dict(torch.load(f'/home/zss/estimation_new/baseline_new/GLSTM/GLSTM_best_model_threshold/best_model_{name}.pth'))
    model_T.eval()
    with torch.no_grad():
        outputs = model_T(features)

    x = outputs[range(len(matrix))].squeeze()

    np.save(f'/home/zss/estimation_new/baseline_new/GLSTM/predict_result/{name}_GLSTM_predict_threshold.npy', x)
